# Simulation notebook

In [1]:
import uuid

# Parameters cells
WITNESS_NAME = None
SIMULATION_PATH = "./simulated_data"
MLFLOW_URL = None
AIRFLOW_DAG_RUN_ID = "local-execution"
AWS_ACCESS_KEY_ID="minio123"
AWS_SECRET_ACCESS_KEY="minio123"
MLFLOW_S3_ENDPOINT_URL="http://localhost:9990"

In [2]:
from os import environ

environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY

In [3]:
from qutip import basis, tensor, rand_ket
import numpy as np

from entanglement_witnesses import witnesses

import mlflow

In [4]:
if MLFLOW_URL is not None:
    environ["MLFLOW_S3_ENDPOINT_URL"] = MLFLOW_S3_ENDPOINT_URL
    mlflow.set_tracking_uri(MLFLOW_URL)

In [5]:
def get_simulated_training_data(entanglement_witness, samples_nb=2000):
    samples_states = []
    samples_is_entangled = []
    
    for _ in range(0, samples_nb):
        # Create an entangled state in the form: cos(theta)|00> + exp(i*phi)*sin(theta)|11>
        theta = np.random.uniform(0, np.pi)
        phi = np.random.uniform(0, 2 * np.pi)
        state = (np.cos(theta) * tensor(basis(2, 0), basis(2, 0)) +
                 np.exp(1j * phi) * np.sin(theta) * tensor(basis(2, 1), basis(2, 1)))
        state_dm = state * state.dag()
        samples_states.append(state_dm)
        is_entangled = entanglement_witness(state_dm)
        samples_is_entangled.append(is_entangled)
    
        # Create an entangled state in the form: cos(theta)|01> + exp(i*phi)*sin(theta)|01>
        theta = np.random.uniform(0, np.pi)
        phi = np.random.uniform(0, 2 * np.pi)
        state = (np.cos(theta) * tensor(basis(2, 0), basis(2, 1)) +
                 np.exp(1j * phi) * np.sin(theta) * tensor(basis(2, 1), basis(2, 0)))
        state = state * state.dag()
        state_dm = state * state.dag()
        samples_states.append(state_dm)
        is_entangled = entanglement_witness(state_dm)
        samples_is_entangled.append(is_entangled)
    
        # Create a separable state |psi> X |phi>
        state = tensor(rand_ket(2), rand_ket(2)).unit()
        state_dm = state * state.dag()
        samples_states.append(state_dm)
        is_entangled = entanglement_witness(state_dm)
        samples_is_entangled.append(is_entangled)

    return samples_states, samples_is_entangled

In [6]:
simulated_data = {}

if WITNESS_NAME is not None:
    parameter_witnesses = {WITNESS_NAME: witnesses[WITNESS_NAME]}
else: 
    parameter_witnesses = witnesses

for name, witness in parameter_witnesses.items():
    samples_states, samples_is_entangled =  get_simulated_training_data(witness)

    simulated_data[name] = {
        "states": samples_states,
        "entanglement": samples_is_entangled
    }

In [10]:
for name, data in simulated_data.items():
    states = data["states"]
    labels = np.array(data["entanglement"])

    flatten_states = [np.concatenate([np.real(state.full()).flatten(), np.imag(state.full()).flatten()]) for state in states]

    file_path = "{}/simulation-{}.npz".format(SIMULATION_PATH, name)
    np.savez(file_path, states=np.array(flatten_states), labels=labels)

    if MLFLOW_URL is not None:
        mlflow.set_experiment('ML Quantum Entanglement')
        with mlflow.start_run() as run:
            mlflow.set_tag("airflow_dag_run_id", AIRFLOW_DAG_RUN_ID)
            mlflow.log_artifact(file_path, artifact_path="simulated_data")